In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model('model_uasaiV2.h5')

In [4]:
labels = {0: 'with_mask', 1: 'incorrect_mask', 2: 'without_mask'}
colors = {0: (0,255,0), 1: (255,255,0), 2: (0,0,255)}

In [5]:
import cv2
import numpy as np
from cv2 import CascadeClassifier

img_size = 128
size = 4
cap = cv2.VideoCapture(0)
classifier = CascadeClassifier('haarcascade_frontalface_default.xml')
while True:
  val, img = cap.read()
  img = cv2.flip(img,1,1)
  
  # resize image
  resized = cv2.resize(img, (img.shape[1] // size, img.shape[0] // size))

  # detect Multiscale/faces
  faces = classifier.detectMultiScale(resized)

  # to draw rectangle
  for f in faces:
    (x,y,w,h) = [v*size for v in f]

    # save rectangle faces
    face_img = img[y:y+h, x:x+w]
    resized = cv2.resize(face_img,(img_size,img_size))
    normalized = resized/255.0
    reshaped = np.reshape(normalized,(1,img_size,img_size,3))
    reshaped = np.vstack([reshaped])
    result = model.predict(reshaped)
    label = np.argmax(result, axis=1)[0]
    cv2.rectangle(img,(x,y),(x+w,y+h),colors[label],2)
    cv2.rectangle(img,(x,y-40),(x+w,y),colors[label],-1)
    cv2.putText(img, labels[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
  cv2.imshow('LIVE', img)
  key = cv2.waitKey(10)
  if key == 27:
    break
cap.release()
cv2.destroyAllWindows()